In [1]:
import imp
import torch
import PIL.Image
import os
import torchvision
import numpy as np
import random

# globals

MainModel = imp.load_source('MainModel', '/home/kylesargent/resnet50_128_pytorch/resnet50_128_pytorch.py') 
model = torch.load('/home/kylesargent/resnet50_128_pytorch/resnet50_128_pytorch.pth')
model.cuda()

DATASET_PATH = '/home/kylesargent/test/'

batch_size = 16
alpha = .01

/home/kylesargent/.local/lib/python2.7/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/kylesargent/.local/lib/python2.7/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [2]:
import load_and_transform
reload(load_and_transform)
from load_and_transform import transform_image, get_image_list, load_image_batch

In [3]:
# data preprocessing

import random

images = get_image_list()
np.random.shuffle(images)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(images[:, 1])

labels = le.transform(images[:, 1])

len(labels)

169396

In [4]:
import torch.nn as nn
from torch.autograd import Variable
cluster_model = nn.Sequential(nn.Linear(128, 128), nn.ReLU(), nn.Linear(128, 128)).cuda()

In [5]:
import torch
from torch.autograd import Variable
torch.cuda.memory_allocated()

95459840L

In [102]:
# training loop
batch_size = 64
from tqdm import tqdm

torch.cuda.empty_cache()
torch.cuda.memory_allocated()

from torch.optim import Adam

alpha = .0001
optimizer = Adam(cluster_model.parameters(), lr=.001)

for i in range(169396 // batch_size):

    batch_labels = torch.from_numpy(labels[i:i + batch_size])
    batch_images = load_image_batch(images[:, 0][i:i + batch_size])

    with torch.no_grad():
        batch_features = model(batch_images)

    batch_features = Variable(batch_features.squeeze())
    batch_codes = cluster_model(batch_features)
    
    M = pairwise_distances(batch_codes, batch_codes)
    
    # delta[i][j] is 1 when code i and code j are the same person
    delta = (batch_labels.unsqueeze(0) - batch_labels.unsqueeze(1)) == 0

    discounts = (delta - alpha).type('torch.FloatTensor').cuda()
    loss = torch.sum(discounts*M)
    
    print(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


tensor(2175.0210, device='cuda:0', grad_fn=<SumBackward0>)
tensor(913.9758, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1053.7426, device='cuda:0', grad_fn=<SumBackward0>)
tensor(591.9983, device='cuda:0', grad_fn=<SumBackward0>)
tensor(878.9588, device='cuda:0', grad_fn=<SumBackward0>)
tensor(528.3573, device='cuda:0', grad_fn=<SumBackward0>)
tensor(3076.2188, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1482.3081, device='cuda:0', grad_fn=<SumBackward0>)
tensor(2327.6030, device='cuda:0', grad_fn=<SumBackward0>)
tensor(2195.5107, device='cuda:0', grad_fn=<SumBackward0>)
tensor(4286.2510, device='cuda:0', grad_fn=<SumBackward0>)
tensor(2856.1843, device='cuda:0', grad_fn=<SumBackward0>)
tensor(4819.1392, device='cuda:0', grad_fn=<SumBackward0>)
tensor(2780.9839, device='cuda:0', grad_fn=<SumBackward0>)
tensor(3071.7852, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1447.4026, device='cuda:0', grad_fn=<SumBackward0>)
tensor(2308.5391, device='cuda:0', grad_fn=<SumBackward0>)
t

tensor(4472.6084, device='cuda:0', grad_fn=<SumBackward0>)
tensor(3793.4851, device='cuda:0', grad_fn=<SumBackward0>)
tensor(3559.1172, device='cuda:0', grad_fn=<SumBackward0>)
tensor(3014.6162, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1457.7827, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1555.8689, device='cuda:0', grad_fn=<SumBackward0>)
tensor(3385.5601, device='cuda:0', grad_fn=<SumBackward0>)
tensor(2695.2573, device='cuda:0', grad_fn=<SumBackward0>)
tensor(2467.9407, device='cuda:0', grad_fn=<SumBackward0>)
tensor(4695.4893, device='cuda:0', grad_fn=<SumBackward0>)
tensor(2461.1450, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1883.0450, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1880.4624, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1474.8407, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1537.7979, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1585.7096, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1408.6433, device='cuda:0', grad_fn=<SumBackward0

KeyboardInterrupt: 

In [ ]:
(1, 95459840L)
(2, 95459840L)
(3, 172530176L)
(x, 11338029568L)
(y, 253794816L)


In [26]:
x = torch.randn(64, 128)

torch.mm(x, x.transpose(0, 1)))

torch.Size([64, 64])

In [80]:
y = torch.Tensor([1,2,4,2])

(y.unsqueeze(0) - y.unsqueeze(1)) == 0

tensor([[1, 0, 0, 0],
        [0, 1, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 1]], dtype=torch.uint8)

In [74]:
def pairwise_distances(x,y):
    x_norms = torch.sum(x*x, dim=1).unsqueeze(1)
    y_norms = torch.sum(y*y, dim=1).unsqueeze(0)    
    return x_norms - 2*torch.mm(x, y.transpose(0, 1)) + y_norms

tensor(632.4157)
tensor(632.4157)
